In [ ]:
%matplotlib inline

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
import m6toolbox

In [ ]:
temp_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/decav/1.00/woa13_decav_t00_01v2.nc'
salt_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/decav/1.00/woa13_decav_s00_01v2.nc'

temp_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/A5B2/0.25/woa13_A5B2_t00_04v2.nc'
salt_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/A5B2/0.25/woa13_A5B2_s00_04v2.nc'

In [ ]:
temp = Dataset(temp_url, 'r')
salt = Dataset(salt_url, 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_w = -25.5

t_sect = temp.variables['t_an'][0,:,:,lon==lon_w].squeeze()
s_sect = salt.variables['s_an'][0,:,:,lon==lon_w].squeeze()

In [ ]:
# empty columns are entirely masked
empty = np.sum(~t_sect.mask, axis=0) == 0
empty[169:] = True # mask above Greenland

In [ ]:
lat = lat[~empty]
t_sect = t_sect[:,~empty]
s_sect = s_sect[:,~empty]

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_w, lat[i])
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

In [ ]:
sa_int = sa_sect
ct_int = ct_sect

In [ ]:
sa_lay = (sa_int[1:,:] + sa_int[:-1,:]) / 2
ct_lay = (ct_int[1:,:] + ct_int[:-1,:]) / 2

In [ ]:
# depths of all interfaces on which observations are present
gr_int = np.ma.array(np.tile(dep.reshape(-1, 1), (1, sa_int.shape[1])), mask=sa_int.mask)

# thicknesses of all layers between interfaces
gr_th  = np.diff(gr_int, axis=0)

# bottom interface at each column
topo = gr_int.max(axis=0)

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

In [ ]:
def remap(h):
    """
    Remap from original climatological grid according to h
    """

    sa_remap = np.empty_like(h)
    ct_remap = np.empty_like(h)

    # remap by columns
    for i in range(h.shape[1]):
        # we need to make sure we deal with unmasking here,
        # otherwise we'll get the fill values for thickness
        # and salt/temp, which would be just a little weird
        sa_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       sa_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        ct_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       ct_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        
    return sa_remap, ct_remap

In [ ]:
def tdma(A, b):
    x = b.copy()
    
    # modify first-row coefficients
    A[0,1,:] /= A[1,0,:]
    x[0,:]   /= A[1,0,:]
    
    # loop down (forward elimination)
    for k in range(1, b.shape[0] - 1):
        m = A[1,k,:] - A[2,k-1,:] * A[0,k,:]
        A[0,k+1,:] /= m
        x[k,:] = (x[k,:] - A[2,k-1,:] * x[k-1,:]) / m
        
    # final element
    x[-1,:] = (x[-1,:] - A[2,-2,:] * x[-2,:]) / (A[1,-1,:] - A[2,-2,:] * A[0,-1,:])
    
    for k in range(b.shape[0] - 2, -1, -1):
        x[k,:] -= A[0,k+1,:] * x[k+1,:]
        
    return x

In [ ]:
def diffuse(z_int, sa, ct, dt, c_surf, c_n2,
            t_grid=3600, d_rho=0.5, d_surf=0):
    """
    Use an implicit diffusivity equation to evolve the grid defined by z_int at timestep dt.
    """
    
    # calculate mid-layer positions
    z_lay = (z_int[1:,:] + z_int[:-1,:]) / 2
    
    # gravity used by gsw
    g = 9.7963
    
    I = z_int.shape[0] - 1 # number of layers
    A = np.zeros((3, I+1, z_int.shape[1])) # diffusion system coefficients

    # iterate over columns
    for i in range(z_int.shape[1]):
        if c_n2 > 0:
            # calculate local buoyancy term at interfaces
            # from temp/salt data at the centre of layers
            n2, z_c = gsw.Nsquared(sa[:,i], ct[:,i], z_lay[:,i])
            # drho_dz term to convert to distance
            dz_r = np.maximum((n2 * 1e4) / g**2, 1e-20)
            # diffusivity coefficient on interfaces
            k_n2_int = dz_r / d_rho
            # interpolate the diffusivity coefficient from interfaces
            # to layers, where they apply in the diffusion equation
            # we have to (linearly) extrapolate into the top and
            # bottom layers, because we don't have the buoyancy frequency at
            # the surface or the very bottom
            f = interp1d(z_c, k_n2_int, bounds_error=False, fill_value="extrapolate")
            k_n2 = np.maximum(f(z_lay[:,i]), 0)
        else:
            k_n2 = np.zeros_like(z_lay[:,i])
        
        # determine total grid coefficient from
        # background term 1/D, where D is local depth
        # surface stretching 1/(d + d_0) for distance
        # from surface d, and factor d_0
        #
        # k_grid = D/tgrid * (c_surf*k_surf + c_n2*k_n2 + c_b*k_b)
        k_grid = (z_int[-1,i] * (c_surf / (d_surf + z_lay[:,i]) + \
                                 c_n2 * k_n2) \
                  + (1 - c_n2 - c_surf)) / t_grid

        # fill in implicit system coefficients
        A[0,2:,i]     = -dt * I**2 * k_grid[1:]
        A[2,:-2,i]    = -dt * I**2 * k_grid[:-1]
        A[1,[0,-1],i] = 1 # boundary conditions
        A[1,1:-1,i]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])

        # solve tridiagonal system
        #z_next[:,i] = solve_banded((1, 1), A, z_int[:,i],
        #                           overwrite_ab=True, check_finite=False)
        
    # solve tridiagonal system for everywhere at once
    z_new = tdma(A, z_int)
    
    h = np.diff(z_new, axis=0)
    h = np.maximum(h, 1e-10)
    h *= z_int[-1,:] / np.sum(h, axis=0)
    
    return np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)

## Testing Diffusivity Terms - Grids
### Uniform

First we define a uniform grid, where layers vanish when they intersect topography.

In [ ]:
n = 75
# uniform thickness from 0 to max topo with n layers
h = np.ones((n,lat.size)) * topo.max() / n

h_i = np.where(h.cumsum(axis=0) > topo)
# deflate all layers below topography
h[h_i] = 1e-3
# get unique latitudes (to get the first cell that needs inflation)
_, i = np.unique(h_i[1], return_index=True)
# inflate
h[h_i[0][i], h_i[1][i]] += (topo - h.sum(axis=0))[h_i[1][i]]

sa, ct = remap(h)

z = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)

In [ ]:
plt.pcolormesh(lat, z, h)
plt.plot(lat, z.T, 'k', linewidth=0.4)
plt.colorbar()
plt.gca().invert_yaxis()

### OM4

We also use the OM4 grid with the `dz_f1` helper function.

In [ ]:
def dz_f1(n, dz_min, total, power, precision):
    dz = np.empty(n)
    
    # initial profile
    for i in range(n):
        dz[i] = (i / (n - 1)) ** power
    
    # rescale to total depth and round to precision
    dz[:] = (total - n*dz_min) * (dz[:] / np.sum(dz))
    dz[:] = np.around(dz[:], decimals=precision)
    
    # adjust bottom
    dz[-1] += total - np.sum(dz[:] + dz_min)
    dz[-1] = np.around(dz[-1], decimals=precision)
    
    dz[:] += dz_min
    
    return dz

In [ ]:
dz_75 = dz_f1(75, 2, 4000, 4.5, 2)
z_75 = np.insert(dz_75.cumsum(), 0, 0)[:,np.newaxis]
z_75_full = np.tile(z_75, (1, sa_sect.shape[1]))

# clip at topography
np.putmask(z_75_full, z_75_full > topo, topo)
# slightly inflate vanished layers (we should adjust the full layer thickness here...)
h_75 = np.maximum(np.diff(z_75_full, axis=0), 1e-10)
# recalculate interfaces
z_75 = np.concatenate((np.zeros((1, h_75.shape[1])), h_75.cumsum(axis=0)), axis=0)
sa_75, ct_75 = remap(h_75)

In [ ]:
plt.pcolormesh(lat, z_75, h_75)
plt.plot(lat, z_75.T, 'k', linewidth=0.4)
plt.colorbar()
plt.gca().invert_yaxis()

# Burchard and Rennau Terms
We can investigate the individual Burchard and Rennau diffusivity terms.

## Buoyancy Frequency
### Uniform

In [ ]:
z_n2 = z.copy()
sa_n2 = sa
ct_n2 = ct

for i in range(10):
    z_n2 = diffuse(z_n2, sa_n2, ct_n2,
                   dt=100, t_grid=3*3600,
                   c_n2=1.0, c_surf=0.0)
    h_n2 = np.diff(z_n2, axis=0)
    sa_n2, ct_n2 = remap(h_n2)

In [ ]:
plt.figure(figsize=(10,6))
plt.pcolormesh(lat, z_n2, h_n2)
plt.plot(lat, z_n2.T, 'w', linewidth=0.5)
plt.colorbar()
plt.title('purely buoyancy diffusivity (uniform IC)')
plt.gca().invert_yaxis()

### OM4

In [ ]:
z_n2_75 = z_75.copy()
sa_n2_75 = sa_75
ct_n2_75 = ct_75

for i in range(10):
    z_n2_75 = diffuse(z_n2_75, sa_n2_75, ct_n2_75,
                      dt=100, t_grid=3*3600,
                      c_n2=1.0, c_surf=0.0)
    h_n2_75 = np.diff(z_n2_75, axis=0)
    sa_n2_75, ct_n2_75 = remap(h_n2_75)

In [ ]:
plt.figure(figsize=(10,6))
plt.pcolormesh(lat, z_n2_75, h_n2_75)
plt.plot(lat, z_n2_75.T, 'w', linewidth=0.5)
plt.colorbar()
plt.title('purely buoyancy diffusivity (OM4 IC)')
plt.gca().invert_yaxis()